# String segmentation

Consider a string that comprises valid words but without any separators. For example,

```python
A = 'considerastringthatcomprisesvalidwordsbutwithoutanyseparators'
```

Can the string be segmented in valid words? Can we write a _boolean_ function to tell us if the string can be segmented in valid words?

As we discussed in class, the answer is yes.

```python
def can_segment(A:str) -> bool:
    n =len(A)
    if n == 0:
        return True
    for i in range(n):
        if is_word(A[:i]):
            if can_segment(A[i:]):
                return True
    return False
```

The code above requires a function, called `is_word` to validate substring `A[:i]` as a word. In a realistic situation, `is_word(s:str)` looks up `s` in a dictionary (the book, not the Python data structure) and returns `True` if found and `False` otherwise. Here, for our work, we'll use a smaller dictionary, in the form of a plain list of strings.


In [1]:
little_dictionary = [
    "a",
    "afternoon",
    "airport",
    "an",
    "and",
    "anywhere",
    "at",
    "because",
    "become",
    "before",
    "behind",
    "beside",
    "between",
    "breakfast",
    "broadcast",
    "but",
    "daylight",
    "deadline",
    "everywhere",
    "for",
    "forecast",
    "handbook",
    "hardware",
    "he",
    "headline",
    "her",
    "him",
    "homework",
    "i",
    "in",
    "income",
    "inside",
    "into",
    "it",
    "keyboard",
    "me",
    "midnight",
    "network",
    "notebook",
    "of",
    "offline",
    "on",
    "online",
    "or",
    "outcome",
    "outline",
    "outside",
    "overcome",
    "oversee",
    "password",
    "saturn",
    "she",
    "software",
    "somewhere",
    "suitcase",
    "sunlight",
    "takeover",
    "textbook",
    "the",
    "them",
    "therefore",
    "they",
    "to",
    "understand",
    "undertake",
    "upon",
    "us",
    "we",
    "whatever",
    "whenever",
    "wherever",
    "whoever",
    "within",
    "without",
    "you",
]

The `little_dictionary` above is sorted so we can search it using binary search.


In [2]:
def binary_search(word_list: list[str], target: str) -> int:
    """Performs binary search on a sorted list of words.
    Returns the index of target if found, otherwise -1.
    To preserve memory, this implementation is iterative."""
    low: int = 0
    high: int = len(word_list) - 1

    while low <= high:
        mid: int = (low + high) // 2
        guess: str = word_list[mid]

        if guess == target:
            return mid  # Target found
        if guess > target:
            high = mid - 1  # Search in the lower half
        else:
            low = mid + 1  # Search in the upper half

    return -1  # Target not found


def is_word(word_list: list[str], word: str) -> bool:
    """Boolean helper for binary searchReturns True if word is in 
    word_list, False otherwise."""
    return binary_search(word_list, word) != -1

We also need a method to produce random strings by concatenating entries from the `little_dictionary` and then distorting them to some degree. For example, the concatenated string `homeworksuitcase` will always be segmentable because it contains entries from the dictionary. We want to see what happens if the string is distorted by dropping a few characters: `homewrksuitase`. The method below delivers suitably distorted strings for our experimentation. Since we are using such a little dictionary, it's important to keep distorion at or below 5%.


In [3]:
import random


def generate_random_string(
        from_words: list[str],
        number_of_words: int,
        distortion_probability: float = 0.0) -> str:
    """Produce a string with randomly selected strings from a list of strings. Then,
    distort the string by removing characters from it with the given probability."""
    # Select words at random from the input list. The number of words to select
    # is specified by the int:number_of_words in the arguments.
    random_string: str = ""
    # Guard statement
    if (distortion_probability >= 0.0
        and number_of_words > 0
            and number_of_words <= len(from_words)):
        random_string = "".join(random.sample(from_words, number_of_words))
        # Distort the string only if the distortion probability is not zero.
        if distortion_probability > 0.0:
            distorted_chars = [
                ch for ch in random_string
                if random.random() >= distortion_probability
            ]
            random_string = "".join(distorted_chars)
    return random_string

This is an implementation of the recursive strategy as described in Chapter 2 of the book.


In [4]:
def can_segment(A: str) -> bool:
    """Determine if a string can be segmented into valid tokens.
    The computation is done recursively"""
    n = len(A)  # shortcut
    if n == 0:
        # Base case: an empty string can be segmented into 0 words
        return True
    for i in range(n):
        # Check if the first i characters of the string are a valid token
        if is_word(little_dictionary, A[:i+1]):
            # Can the rest of the string be segmented
            if can_segment(A[i+1:]):
                return True
    return False

A brief demo checks 10 randomly created strings each concatenating 15 words from the little dictionary, with distortion probability 1%.


In [17]:
number_of_trials = 10
number_of_words = 15
distortion_probability = 0.01
current_trial = 0

while current_trial < number_of_trials:
    current_trial += 1
    A = generate_random_string(
        little_dictionary, number_of_words, distortion_probability)
    result: bool = can_segment(A)
    # report
    print(f"{str(result):>8}: {A}")

   False: betweenibeforedeadlinehedlinedaylightofflinebroadcastfornotebookitafternoonwhateverwherevrtherefore
   False: everwherewhoeverwhateverherasuitcaseoutcomeoftheyinforecastwhereveruponundertakeairport
   False: anywherbecomeoutsidebesidewhatevermeunderstndinsideiorhoeworkwehedaylightairport
    True: anywherethesuitcasebehindoverseewhatevereverywherewewithoutkeyboardbecomemidnighthimbetweentakeover
    True: theanbesideairportwhereverhardwaremeofflineanywheresaturnincomeusmidnightyounotebook
   False: outlineoutsidehardwareoverseeansheincomewithoutbesideoutcomenetworktheweafternooovercome
   False: olineforwheneveroutlinehardwaresutcaseshebetweenbecausebehindsoftwareatteincomewithout
    True: besidewithinwhoeverwithoutonustheyovercomebecauseoutsidebroadcastundertakenotebookafternoonhandbook
   False: youheadlneintooutlinesuitcasebeforewithoutafternoonpassworditeverywerebutsoftwareonoutside
   False: betweenonlinemebesideovercmetextbookhandokitonbecauseintoininsidekeyboardsunlig

The cool thing about recursion is that it looks cool -- a function calling itself! Practically, however, there is nothing cool about recursion. It takes up too much memory, and also slows things down. (Just try to compute, recursively, large Fibonnaci values. See appendix at the end of this notebook.) Anytime recursion becomes an obstacle, there may be a dynamic programming solution. 

Indeed, we can determine the segmentability of a string of length $n$ by asking if its $n$-th prefix is segmentable. For example, consider the (obviously segmentable) string 
```python
A = "inaholeinthegroundtherelivedahobbit"
```
that has 35 characters. Its dynamic programming array is defined as:
$$
\texttt{dp[i]} = \begin{cases}\texttt{True}\ \text{if}\ \texttt{A[0:i]}\ \text{is segmentable}\\ \texttt{False}\ \text{otherwise}\end{cases}
$$
with `dp[0]=True` by definition (the empty string can be segmented into 0 words). The rest of the array will go like this if we have access to a substantive dictionary (not just the little dictionary from earlier):

* `dp[1] = True` because `"i"` is segmentable (to the word *i);*
* `dp[2] = True` because `"in"` is segmentable (to the word *in);*
* `dp[3] = True` because `"ina"` is segmentable (to the words *in* and *a);*
* `dp[4] = False` because `"inah"` cannot be segmented to valid words; etc.

The method below builds on the prior knowledge stored in `dp`. For example we know that `ina` is segmentable because `in` is segmentable (thank you `dp[2]` and `A[1:2]` (=`a`) is a valid word).


In [ ]:
def can_segment_dp(A: str) -> bool:
    """Determine (using dynamic programming) if A can be segmented into
    valid tokens from little_dictionary."""
    # Shortuct
    n: int = len(A)
    # Initialize the dp array
    dp: list[bool] = [False] * (n + 1)
    # Base case
    dp[0] = True

    # Consider every prefix A[:i] for i in 1..n
    for i in range(1, n + 1):
        j = 0
        # we continue until we either find a valid split or exhaust j
        while j < i and not dp[i]:
            if dp[j] and is_word(little_dictionary, A[j:i]):
                dp[i] = True
            j += 1

    return dp[n]



In [ ]:
# test
number_of_trials = 10
current_trial = 0
distortion_probability = 0.01
while current_trial < number_of_trials:
    current_trial += 1
    A = generate_random_string(
        little_dictionary, number_of_trials, distortion_probability)
    result: bool = can_segment_dp(A)
    # report
    print(f"{str(result):>8}: {A}")

Of course, the method above only tells us if a string is segmentable or not. It does not provide the segmentation.

# Assignment
Modify the method above to return the segmentation of a text (if segmentable) or `None` (otherwise).

For example
```python
can_segment_dp("anywherethesuitcasebehindoversee")
```
should return
```python
['anywhere', 'the', 'suitcase', 'behind', 'oversee']
```

---

# Appendix: Fibonacci


In [ ]:
def fibo_recursive(n):
    if n == 0 or n == 1:
        return n
    else:
        return fibo_recursive(n-1)+fibo_recursive(n-2)


print(fibo_recursive(10))

55


In [ ]:

def fibo_iterative(n):
    previous = 1
    previous_previous = 0
    if n == 0:
        return previous_previous
    elif n == 1:
        return previous
    else:
        count = 1
        while count < n:
            next = previous + previous_previous
            previous_previous = previous
            previous = next
            count += 1
        return next


print(fibo_iterative(10))

55


In [45]:
# Compare
import time
billion = 1_000_000_000
n = 2
cutoff = 128
print(f"{"n":>3s}\t{"Recursive":>15s}\t{"Iterative":>15s}")
print(f"{" ":>3s}\t{"(seconds)":>15s}\t{"(seconds)":>15s}\n")
while n < cutoff:
    start = time.time_ns()
    fib = fibo_recursive(n)
    stop = time.time_ns()
    t_rec = (stop-start) / billion
    start = time.time_ns()
    fib = fibo_iterative(n)
    stop = time.time_ns()
    t_ite = (stop-start) / billion
    # report
    print(f"{n:3}\t{t_rec:15.11f}\t{t_ite:15.11f}")
    # next n
    n *= 2

  n	      Recursive	      Iterative
   	      (seconds)	      (seconds)

  2	  0.00000284500	  0.00000191300
  4	  0.00000197400	  0.00000125200
  8	  0.00000696300	  0.00000110200
 16	  0.00026364100	  0.00000226400
 32	  0.35072740200	  0.00000373700


KeyboardInterrupt: 